## Named entity recognition with spacy

In [17]:
import spacy
import pandas as pd
import numpy as np
from nltk.stem import SnowballStemmer
from collections import Counter

# nlp = spacy.load('en_core_web_sm')
# nlpmd = spacy.load('en_core_web_md')
nlplg = spacy.load('en_core_web_lg')


In [42]:
# Import the email modules we'll need
import glob
import email
import mailparser
from email import policy
from email.parser import BytesParser

path = '../datawe/raw/Email_Classification/*'
email_types = glob.glob(path)
appendFilesData = []
file_raw_data = [] 
for folder in email_types:
    files = glob.glob(folder+"/*.txt")
    email_type = folder.split('\\')[1]
    for name in files:
        try:
            with open(name) as fp:
                raw_data = fp.read()
                file_raw_data.append(raw_data)
                msg = mailparser.parse_from_string(raw_data)
                appendFilesData.append({
                    "to":msg.to,
                    "from":msg.from_,
                    "subject":msg.subject,
                    "date":msg.date,
#                     "sent":msg["Sent"],
#                     "importance":msg["Importance"],
                    "content": raw_data, # msg.body,
                    "class_to_exec":email_type,
                })
         
        except IOError as exc:
            print('Exception')

            
#creating pandas dataframe
data = pd.DataFrame(appendFilesData)

In [46]:
# data
data["content"] = data["content"].apply(lambda x: x.replace('\n', ' '))
# data['content']

In [47]:
body = data["content"].values
entityData = []
for cnt in body:
    entityCh = []
    entity = nlplg(cnt)
    for token in entity.ents:
        entityData.append({"name":token.text, "label":token.label_, "end_char":token.end_char,"start_char":token.start_char,"text":cnt})


#     entityData.append(entityCh)


In [48]:
entityData

[{'name': 'December 2018',
  'label': 'DATE',
  'end_char': 72,
  'start_char': 59,
  'text': 'Please book a room at lake palace  Check in date is - 31st December 2018 check out - 1st January 2019'},
 {'name': 'January 2019',
  'label': 'DATE',
  'end_char': 101,
  'start_char': 89,
  'text': 'Please book a room at lake palace  Check in date is - 31st December 2018 check out - 1st January 2019'},
 {'name': 'December 2018',
  'label': 'DATE',
  'end_char': 72,
  'start_char': 59,
  'text': 'Kindly book a room at lake palace  Check in date is - 31st December 2018 check out - 1st January 2019'},
 {'name': 'January 2019',
  'label': 'DATE',
  'end_char': 101,
  'start_char': 89,
  'text': 'Kindly book a room at lake palace  Check in date is - 31st December 2018 check out - 1st January 2019'},
 {'name': '  ',
  'label': 'ORG',
  'end_char': 2,
  'start_char': 0,
  'text': '  Dear Rohit,   As per our conversation a while ago, please book and confirm 02 Executive room at Taj Gateway Ganges fr

In [49]:
entityDF = pd.DataFrame(entityData)

In [50]:
entityDF.head()

,end_char,label,name,start_char,text
0,72,DATE,December 2018,59,Please book a room at lake palace Check in da...
1,101,DATE,January 2019,89,Please book a room at lake palace Check in da...
2,72,DATE,December 2018,59,Kindly book a room at lake palace Check in da...
3,101,DATE,January 2019,89,Kindly book a room at lake palace Check in da...
4,2,ORG,,0,"Dear Rohit, As per our conversation a whil..."


In [51]:
entityDF.to_csv("C:/Users/Lokesh/jupyter works/email_classification/datawe/raw/Email_Classification/email_entity_train_Lake.csv")

In [29]:
type(entityDF["text"][1])

str

In [30]:
# text = nlp('. '.join(np.array(data["content"])))

In [31]:
# for token in text:
#     print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#           token.shape_, token.is_alpha, token.is_stop)

In [32]:
# entityDtl = []
# for token in text.ents:
#     if len(token.text.rstrip()) > 0:
#         entityDtl.append({"label":token.label_, "text":token.text})

In [33]:
# entyDF = pd.DataFrame(entityDtl)
# entyDF

In [34]:
# type(entyDF["text"][3])

In [35]:
# import spacy
# nlp = spacy.load("en_core_web_sm")

In [36]:
sentence = """
Dear Sir/ Madam,

 

Greetings of the day!

 

As per your request, I am pleased to assist you with the reservation
details as follows:

 

*         Reservation #: 75677SB034081

*         Guest Name: Saroj Kumar

*         Hotel: Taj Dubai

*         Dates of Stay: Tue Nov 20 - Sun Nov 25, 2018

*         Room: Luxury Burj View Room King Bed

*         Rate: Stay a Bit Longer – AED 6500.61

*         Inclusions: Breakfast, complimentary shuttle to Dubai mall at
fixed timings and Wi-Fi upto 4 devices 

*         Cancellation: Reservations must be cancelled by 2PM - 1 day
prior to arrival to avoid a penalty of 1 night charge plus taxes

 

Please do not hesitate to contact us if we may be of further assistance
to you.

 

Warm Regards,

Diana Chettiar

Taj Reservations Worldwide Team





 """

In [37]:


# # en_core_web_sm = spacy.load('en_core_web_sm')
# # en_core_web_md = spacy.load('en_core_web_md')
# # en_core_web_lg = spacy.load('en_core_web_lg')
# # en_vectors_web_lg = spacy.load('en_vectors_web_lg')



# # en_core_web_sm_doc = en_core_web_sm(sentence)
# # en_core_web_md_doc = en_core_web_md(sentence)
# # en_core_web_lg_doc = en_core_web_lg(sentence)
# # en_vectors_web_lg_doc = en_vectors_web_lg(sentence)

# doc = nlp(sentence)

# entityDtl = []
# for token in text.ents:
#     if len(token.text.rstrip()) > 0:
#         entityDtl.append({"label":token.label_, "text":token.text})

In [38]:
# pd.DataFrame(entityDtl)

entities = {"GPE":"Location",
 "ORG":"organization",
 "PERSON":"Name",
 "DATE":"Date",
 "TIME" : "Time",
 "CARDINAL": "CARDINAL"
}
stemmer = SnowballStemmer("english").stem

In [40]:
def findEntities(text):
    entityDtl = []
    for token in text.ents:
        wrd = token.text.strip()
        label_ =  token.label_
        if len(wrd) > 0:
            entityDtl.append({"label":token.label_, "text":token.text.strip()})

#     if len(wrd) > 0 and label_ in entities and ((label_ == "PERSON" and wrd.replace(' ','').isalpha() and stemmer(wrd) == wrd.lower()) or label_ != "PERSON"):
#         entityDtl.append({"label":token.label_, "text":token.text.strip()})

    return pd.DataFrame(entityDtl)

In [41]:
textlg = nlplg(sentence)
textmd = nlpmd(sentence)
entityLg = findEntities(textlg)
entityMd = findEntities(textmd)

In [21]:
# names = entityDtlDF[entityDtlDF["label"] == "PERSON"]["text"]

# # names.

# stemmer = SnowballStemmer("english").stem
# stem_free = [stemmer(stem) for stem in names if stem]
# stem_free
items = [x.text for x in text.ents]
Counter(items).most_common(3)

[('\n\n', 5), ('75677SB034081', 1), ('Kumar\n\n', 1)]

In [42]:
entityLg, entityMd

(       label                             text
 0   CARDINAL                    75677SB034081
 1     PERSON                            Kumar
 2        GPE                        Taj Dubai
 3       DATE         Tue Nov 20 - Sun Nov 25,
 4       DATE                             2018
 5        FAC            Luxury Burj View Room
 6        ORG            Inclusions: Breakfast
 7        GPE                            Dubai
 8        FAC                            Wi-Fi
 9       DATE                      2PM - 1 day
 10      TIME                          1 night
 11    PERSON                   Diana Chettiar
 12       ORG  Taj Reservations Worldwide Team,
        label                                               text
 0     PERSON                                         Sir/ Madam
 1   CARDINAL                                      75677SB034081
 2     PERSON                                              Guest
 3     PERSON                                        Saroj Kumar
 4     PERSON   

In [221]:
((label_ == "PERSON" and wrd.isalpha() and stemmer(wrd) == wrd) or label_ != "PERSON")

False

In [225]:
stemmer(wrd) == wrd.lower()

True

In [226]:
wrd.isalpha()

False

In [224]:
stemmer(wrd)

'diana chettiar'

In [130]:
# import spacy

# nlp = spacy.load('en_core_web_md')  # make sure to use larger model!
tokens = nlp(u'dog cat banana')

for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

dog dog 1.0
dog cat 0.53906965
dog banana 0.28761008
cat dog 0.53906965
cat cat 1.0
cat banana 0.4875216
banana dog 0.28761008
banana cat 0.4875216
banana banana 1.0


In [215]:
help("")

Help on class str in module builtins:

class str(object)
 |  str(object='') -> str
 |  str(bytes_or_buffer[, encoding[, errors]]) -> str
 |  
 |  Create a new string object from the given object. If encoding or
 |  errors is specified, then the object must expose a data buffer
 |  that will be decoded using the given encoding and error handler.
 |  Otherwise, returns the result of object.__str__() (if defined)
 |  or repr(object).
 |  encoding defaults to sys.getdefaultencoding().
 |  errors defaults to 'strict'.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __format__(...)
 |      S.__format__(format_spec) -> str
 |      
 |      Return a formatted version of S as described by format_spec.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getatt

In [3]:
import spacy

nlp = spacy.load('en_core_web_sm')
doc = nlp(u'San Francisco considers banning sidewalk delivery robots')
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

from spacy.tokens import Span
doc = nlp(u'FB is hiring a new VP of global policy')
doc.ents = [Span(doc, 0, 1, label=doc.vocab.strings[u'ORG'])]
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

San Francisco 0 13 GPE
FB 0 2 ORG


In [43]:
import spacy

nlp = spacy.load('en_core_web_lg')
doc = nlp(u""+sentence)
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
          chunk.root.head.text)


Dear Sir/ Madam Madam ROOT Madam
Greetings Greetings appos Madam
the day day pobj of
your request request pobj per
I I nsubj am
you you dobj assist
the reservation
details details pobj with
*         Reservation # # ROOT #
*         Guest Name Name ROOT Name
Saroj Kumar Kumar appos Name
*         Hotel Hotel appos Kumar
Taj Dubai Dubai appos Hotel
*         Dates Dates ROOT Dates
Stay Stay pobj of
*         Room Room ROOT Room
Luxury Burj View Room Room appos Room
King Bed Bed ROOT Bed
*         Rate Rate ROOT Rate
a Bit Longer – – dobj Stay
: Breakfast Breakfast appos Inclusions
complimentary shuttle shuttle appos Breakfast
Dubai mall mall pobj to
fixed timings timings appos Breakfast
Wi-Fi Fi conj Breakfast
4 devices devices dobj upto
*         Cancellation Cancellation appos devices
Reservations Reservations nsubjpass cancelled
2PM - 1 day day pobj by
arrival arrival pobj to
a penalty penalty dobj avoid
1 night charge charge pobj of
taxes taxes conj charge
us us dobj contact
we we 

In [45]:
import spacy
from spacy.tokens import Span

nlp = spacy.load('en_core_web_lg')
doc = nlp(u"FB is hiring a new Vice President of global policy")
ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print('Before', ents)
# the model didn't recognise "FB" as an entity :(

ORG = doc.vocab.strings[u'ORG']  # get hash value of entity label
fb_ent = Span(doc, 0, 1, label=ORG) # create a Span for the new entity
doc.ents = list(doc.ents) + [fb_ent]

ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print('After', ents)

Before []
After [('FB', 0, 2, 'ORG')]


In [49]:
doc = nlp.make_doc(u'This is a sentence')   # create a Doc from raw text
for name, proc in nlp.pipeline:             # iterate over components in order
    doc = proc(doc)     
    print(name, proc)

tagger <spacy.pipeline.Tagger object at 0x000001C2CCDD5630>
parser <spacy.pipeline.DependencyParser object at 0x000001C2CCD10830>
ner <spacy.pipeline.EntityRecognizer object at 0x000001C2CCD10780>


In [47]:
doc

This is a sentence

In [57]:
sentence = []
for sent in textlg.sents:
    sentence.append(sent)

sentence[5]

*         Room: Luxury Burj View Room

In [3]:
#!/usr/bin/python3

# unicode.py

text = u'\u041b\u0435\u0432 \u041d\u0438\u043a\u043e\u043b\u0430\ \n\
\u0435\u0432\u0438\u0447 \u0422\u043e\u043b\u0441\u0442\u043e\u0439: \n\
\u0410\u043d\u043d\u0430 \u041a\u0430\u0440\u0435\u043d\u0438\u043d\u0430'

print(text)

Лев Никола\ 
евич Толстой: 
Анна Каренина


In [4]:



# "PERSON":	People, including fictional.
# NORP	Nationalities or religious or political groups.
# FAC	Buildings, airports, highways, bridges, etc.
# ORG	Companies, agencies, institutions, etc.
# GPE	Countries, cities, states.
# LOC	Non-GPE locations, mountain ranges, bodies of water.
# PRODUCT	Objects, vehicles, foods, etc. (Not services.)
# EVENT	Named hurricanes, battles, wars, sports events, etc.
# WORK_OF_ART	Titles of books, songs, etc.
# LAW	Named documents made into laws.
# LANGUAGE	Any named language.
# DATE	Absolute or relative dates or periods.
# TIME	Times smaller than a day.
# PERCENT	Percentage, including "%".
# MONEY	Monetary values, including unit.
# QUANTITY	Measurements, as of weight or distance.
# ORDINAL	"first", "second", etc.
# CARDINAL	Numerals that do not fall under another type.


In [15]:
import pandas as pd
import numpy as np
import spacy
import random
from spacy.util import minibatch, compounding

class spacy_train_model:
    def __init__(self):
       # self.nlp = spacy.load('en_core_web_lg')
        df = pd.read_csv("C:/Users/Lokesh/jupyter works/email_classification/datawe/raw/Email_Classification/small_train.csv")
        entityDf = df[df.apply(lambda x: x["text"][x["start_char"]:x["end_char"]] == x["Entity"], axis=1)]
        entityTrainData = []
        for text, item in  entityDf['text'].value_counts().items():
            mulItems = entityDf[entityDf['text'] == text]
            multipleEntities = []
            for dta in mulItems.values:
                multipleEntities.append((dta[3], dta[1], dta[2]))
            entityTrainData.append((text, {'entities':multipleEntities}))

        self.trainedModel = self.main_train(model = 'en_core_web_lg', TRAIN_DATA = entityTrainData)
   

    def main_train(self, model=None, output_dir=None, n_iter=100, TRAIN_DATA = []):
        """Load the model, set up the pipeline and train the entity recognizer."""
        if model is not None:
            nlp = spacy.load(model)  # load existing spaCy model
#             print("Loaded model '%s'" % model)
        else:
            nlp = spacy.blank('en')  # create blank Language class
#             print("Created blank 'en' model")

        # create the built-in pipeline components and add them to the pipeline
        # nlp.create_pipe works for built-ins that are registered with spaCy
        if 'ner' not in nlp.pipe_names:
            ner = nlp.create_pipe('ner')
            nlp.add_pipe(ner, last=True)
        # otherwise, get it so we can add labels
        else:
            ner = nlp.get_pipe('ner')
#         print(TRAIN_DATA)

        # add labels
        for _, annotations in TRAIN_DATA:
            for ent in annotations.get('entities'):
                ner.add_label(ent[2])

        # get names of other pipes to disable them during training
        other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
        with nlp.disable_pipes(*other_pipes):  # only train NER
            optimizer = nlp.begin_training()
            for itn in range(n_iter):
                random.shuffle(TRAIN_DATA)
                losses = {}
                # batch up the examples using spaCy's minibatch
                batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
                for batch in batches:
                    texts, annotations = zip(*batch)
                    print(texts)
                    print(annotations)
                    nlp.update(
                        texts,  # batch of texts
                        annotations,  # batch of annotations
                        drop=0.5,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
#                 print('Losses', losses)

        # test the trained model
        for text, _ in TRAIN_DATA:
            doc = nlp(text)
#             print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#             print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

        # save model to output directory
        if output_dir is not None:
            output_dir = Path(output_dir)
            if not output_dir.exists():
                output_dir.mkdir()
            nlp.to_disk(output_dir)
#           print("Saved model to", output_dir)

            # test the saved model
#            print("Loading from", output_dir)
            nlp2 = spacy.load(output_dir)
            for text, _ in TRAIN_DATA:
                doc = nlp2(text)
#                print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#                print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])
        return nlp
    # if __name__ == "__main__":
    #     main_train('en_core_web_lg')




In [16]:
train_model = spacy_train_model()

('check in and check out date 31/12/2018, 1/1/2019', 'i want to book a room at lake palace. Check in date is - 20th december 2018, check out - 8th January 2019', 'Dear Rohit,   As per our conversation a while ago, please book and confirm 02 Executive room at Taj Gateway Ganges from 24/26 December 2018 for two nights.', 'i want to book a room at lake palace. Check in date is - 31st December 2018, check out - 1st January 2019')
({'entities': [('DATE', '31/12/2018', 38), ('DATE', '1/1/2019', 48)]}, {'entities': [('DATE', '8th January 2019', 105)]}, {'entities': [('PERSON', 'Rohit', 10), ('ORG', 'Taj Gateway Ganges', 114), ('DATE', '24/26 December 2018', 139), ('DATE', 'two nights', 154)]}, {'entities': [('LOC', 'lake palace', 36), ('DATE', '31st December 2018', 75)]})


TypeError: 'str' object cannot be interpreted as an integer